# 1. Environment setup

In [1]:
!pip install -q transformers datasets scikit-learn peft accelerate bitsandbytes


In [2]:
import pandas as pd
import torch
import numpy as np
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from datasets import Dataset
from transformers import (
    DistilBertTokenizerFast,
    DistilBertForSequenceClassification,
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments
)
from peft import LoraConfig, get_peft_model


# 2. Dataset Loading & Preparation

In [3]:
df=pd.read_csv('/content/reviews.csv',encoding='latin-1',engine='python')
df = df.dropna(subset=["Text", "Sentiment"])
df["Text"] = df["Text"].astype(str)

print("Dataset shape:", df.shape)
print(df["Sentiment"].value_counts())


Dataset shape: (50000, 2)
Sentiment
0    25000
1    25000
Name: count, dtype: int64


# 3. Train–Validation Split

In [4]:
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df["Text"].tolist(),
    df["Sentiment"].tolist(),
    test_size=0.3,
    random_state=42,
    stratify=df["Sentiment"]
)


# MODEL 1 — Full Fine-Tuning (DistilBERT)

4. Tokenization

In [5]:
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

train_enc = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
val_enc   = tokenizer(val_texts, truncation=True, padding=True, max_length=128)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


# 5. Dataset Conversion

In [6]:
train_ds = Dataset.from_dict({
    "input_ids": train_enc["input_ids"],
    "attention_mask": train_enc["attention_mask"],
    "labels": train_labels
})

val_ds = Dataset.from_dict({
    "input_ids": val_enc["input_ids"],
    "attention_mask": val_enc["attention_mask"],
    "labels": val_labels
})

# 6. Model Initialization

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

full_model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2
).to(device)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# 7. Training Configuration

In [8]:
full_args = TrainingArguments(
    output_dir="./full_ft_results",
    num_train_epochs=2,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=5e-5,
    weight_decay=0.01,
    eval_strategy="epoch",
    logging_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    report_to="none"
)


# 8. Metrics Function

In [9]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds)
    }


# 9. Full Fine-Tuning

In [10]:
full_trainer = Trainer(
    model=full_model,
    args=full_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

full_output = full_trainer.train()
full_results = full_trainer.evaluate()


/tmp/ipython-input-491753130.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  full_trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.341000,0.289391,0.873200,0.868936
2,0.180500,0.340654,0.891067,0.890585


# 10. Full Fine-Tuned Prediction (with Confidence)

In [11]:
def predict_full(texts):
    enc = tokenizer(texts, return_tensors="pt", truncation=True, padding=True, max_length=128)
    enc = {k: v.to(device) for k, v in enc.items()}

    with torch.no_grad():
        outputs = full_model(**enc)
        probs = torch.softmax(outputs.logits, dim=-1)

    preds = torch.argmax(probs, dim=-1)
    return preds.cpu().numpy(), probs.max(dim=1).values.cpu().numpy()


# MODEL 2 — LoRA Fine-Tuning

# 11. Dataset Preparation for LoRA

In [12]:
train_df, val_df = train_test_split(
    df,
    test_size=0.3,
    stratify=df["Sentiment"],
    random_state=42
)

train_ds_lora = Dataset.from_pandas(train_df)
val_ds_lora   = Dataset.from_pandas(val_df)


# 12. Tokenization (LoRA)

In [13]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize(batch):
    return tokenizer(batch["Text"], truncation=True, padding="max_length", max_length=128)

train_ds_lora = train_ds_lora.map(tokenize, batched=True)
val_ds_lora   = val_ds_lora.map(tokenize, batched=True)

train_ds_lora = train_ds_lora.rename_column("Sentiment", "labels")
val_ds_lora   = val_ds_lora.rename_column("Sentiment", "labels")

train_ds_lora.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
val_ds_lora.set_format("torch", columns=["input_ids", "attention_mask", "labels"])


Map:   0%|          | 0/35000 [00:00<?, ? examples/s]

Map:   0%|          | 0/15000 [00:00<?, ? examples/s]

# 13. LoRA Model Setup

In [14]:
lora_base = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2
)

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_lin", "v_lin"],
    lora_dropout=0.1,
    bias="none",
    task_type="SEQ_CLS"
)

lora_model = get_peft_model(lora_base, lora_config)
lora_model.print_trainable_parameters()


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 739,586 || all params: 67,694,596 || trainable%: 1.0925


# 14. LoRA Training Configuration

In [15]:
lora_args = TrainingArguments(
    output_dir="./lora_results",
    num_train_epochs=2,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=2e-4,
    fp16=True,
    eval_strategy="epoch",
    logging_strategy="epoch",
    save_strategy="epoch",
    report_to="none"
)


# 15. LoRA Training

In [16]:
lora_trainer = Trainer(
    model=lora_model,
    args=lora_args,
    train_dataset=train_ds_lora,
    eval_dataset=val_ds_lora,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

lora_output = lora_trainer.train()
lora_results = lora_trainer.evaluate()


/tmp/ipython-input-200071499.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  lora_trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.357900,0.309007,0.866467,0.869350
2,0.298300,0.304821,0.871067,0.873197


# 16. LoRA Prediction Function

In [17]:
def predict_lora(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    inputs = {k: v.to(lora_model.device) for k,v in inputs.items()}

    with torch.no_grad():
        outputs = lora_model(**inputs)
        probs = torch.softmax(outputs.logits, dim=1)

    pred = torch.argmax(probs).item()
    confidence = probs[0][pred].item()
    return pred, confidence


# Model Comparision

In [18]:
# Full Fine-Tuning Time
full_train_time = full_output.metrics["train_runtime"]
full_acc = full_results["eval_accuracy"]
full_f1 = full_results["eval_f1"]

# LoRA training time
lora_train_time = lora_output.metrics["train_runtime"]
lora_acc = lora_results["eval_accuracy"]
lora_f1 = lora_results["eval_f1"]

In [19]:
def count_trainable_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def count_total_params(model):
    return sum(p.numel() for p in model.parameters())

full_trainable = count_trainable_params(full_model)
full_total = count_total_params(full_model)

lora_trainable = count_trainable_params(lora_model)
lora_total = count_total_params(lora_model)




In [20]:
# Side-by-Side Comparison
print("\n========== MODEL COMPARISON SUMMARY ==========\n")

print("FULL FINE-TUNING (DistilBERT)")
print("---------------------------------")
print(f"Train Time           : {full_train_time/60:.2f} minutes")
print(f"Validation Accuracy  : {full_acc:.4f}")
print(f"Validation F1 Score  : {full_f1:.4f}")
print(f"Trainable Params     : {full_trainable:,}")
print(f"Total Params         : {full_total:,}")
print()

print("LoRA FINE-TUNING")
print("---------------------------------")
print(f"Train Time           : {lora_train_time/60:.2f} minutes")
print(f"Validation Accuracy  : {lora_acc:.4f}")
print(f"Validation F1 Score  : {lora_f1:.4f}")
print(f"Trainable Params     : {lora_trainable:,}")
print(f"Total Params         : {lora_total:,}")
print()

print("EFFICIENCY GAINS")
print("---------------------------------")
print(f"Speedup Factor       : {full_train_time / lora_train_time:.2f}x faster")
print(f"Parameter Reduction  : {(1 - lora_trainable/full_trainable)*100:.2f}% fewer trainable params")

print("\n=============================================\n")



========== MODEL COMPARISON SUMMARY ==========

FULL FINE-TUNING (DistilBERT)
---------------------------------
Train Time           : 14.92 minutes
Validation Accuracy  : 0.8911
Validation F1 Score  : 0.8906
Trainable Params     : 66,955,010
Total Params         : 66,955,010

LoRA FINE-TUNING
---------------------------------
Train Time           : 4.84 minutes
Validation Accuracy  : 0.8711
Validation F1 Score  : 0.8732
Trainable Params     : 739,586
Total Params         : 67,694,596

EFFICIENCY GAINS
---------------------------------
Speedup Factor       : 3.08x faster
Parameter Reduction  : 98.90% fewer trainable params




# Sample test

In [21]:
test_text = """First thing first, I am not a Salman hater. I went into this movie expecting another good experience, because that's exactly what Bajrangi Bhaijaan did for me. I was disappointed.

This movie means well and wants us love our neighbors, but when the main man of the movie (this movie is 95% Salman) is unconvincing and at times, annoying, you just cant enjoy the movie. I liked what Salman did in Bajrangi. I didn't like what Salman did in this. I cant make up my mind on if he was trying too hard to be a convincing disabled childlike-man or not trying at all, because most of the times the camera was on Salman, he's making extremely silly and at times annoying distorted faces. I know kids and people with similar disabilities do that too, but what Salman was doing looked like he was making fun of such people, far from convincing us that he was one of them. Then again, making Salman play a disabled childlike man was never a good idea.

This movie can get emotional (trying too hard at times) so you might want to bring tissues along. The plot is pretty simple, Laxman aka Tubelight does little but wholesome things to increase his 'yakeen' which he believes will bring his brother back from war. The cinematography is breathtaking, but the war scenes were disappointing and poorly choreographed. The film's music was thoroughly enjoyable, especially Nach Meri Jaan and Radio.

The supporting cast was pretty good. The late Om Puri showed us his excellence for one last time, acing the role as Banne Chacha. Sohail Khan played his part well, nothing else to it. Zhu Zhu was great whenever she was great on-screen, so was little Matin Rey Tangu. Zeeshan Ayyub was convincingly good in his role. SRK's cameo was (no pun intended) magical."""


# Full FT
p_full, c_full = predict_full([test_text])

# LoRA
p_lora, c_lora = predict_lora(test_text)

print("CONFIDENCE COMPARISON (Same Review)")
print("----------------------------------")
print(f"Full Fine-Tuning : {'POSITIVE' if p_full[0]==1 else 'NEGATIVE'} | Confidence: {c_full[0]:.4f}")
print(f"LoRA Fine-Tuning : {'POSITIVE' if p_lora==1 else 'NEGATIVE'} | Confidence: {c_lora:.4f}")

CONFIDENCE COMPARISON (Same Review)
----------------------------------
Full Fine-Tuning : NEGATIVE | Confidence: 0.9905
LoRA Fine-Tuning : NEGATIVE | Confidence: 0.9578


In [22]:
neg_rev="I was absolutely mesmerized by the cinematography—the entire film felt like a dream, but one of those dreams where you're perpetually searching for something you can't quite name. The two-hour runtime was a masterclass in patience testing, proving that the writers are true masters of the drawn-out slow burn. Every single scene was remarkably consistent in its ability to underwhelm. The twist, when it finally arrived, was so brilliantly foreshadowed by a complete lack of context that I can only conclude the director's goal was ambitious, baffling boredom. I left the theatre feeling lighter, having successfully shed the weight of high expectations. Go see it."

# Full FT
p_full, c_full = predict_full([neg_rev])

# LoRA
p_lora, c_lora = predict_lora(neg_rev)

print("CONFIDENCE COMPARISON (Same Review)")
print("----------------------------------")
print(f"Full Fine-Tuning : {'POSITIVE' if p_full[0]==1 else 'NEGATIVE'} | Confidence: {c_full[0]:.4f}")
print(f"LoRA Fine-Tuning : {'POSITIVE' if p_lora==1 else 'NEGATIVE'} | Confidence: {c_lora:.4f}")

CONFIDENCE COMPARISON (Same Review)
----------------------------------
Full Fine-Tuning : POSITIVE | Confidence: 0.7283
LoRA Fine-Tuning : POSITIVE | Confidence: 0.7267
